# Launch jobs
Code to create a batch script for launching jobs on cori
Sep 1, 2020

In [1]:
import os
import glob
import time
import subprocess as sp
import numpy as np

In [2]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/run_scripts


In [3]:
img_size=512
code_dirs={'128':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code/',
         '512':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/512_sqr/'}


results_loc='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size)

In [4]:
# ls /global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20200901_200005_bsize50_30eph/chkpt/trainer0/sgd.shared.training.epoch.30.step.13500/model0/

In [5]:
# glob.glob(results_loc+'*')

In [41]:
### Default dictionary
dict_pars={'job_name':'gen_img', 'time':'00:15:00', 'procs':8,
          'seed':282,'batchsize':200}

# data_dir=results_loc+''
# data_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20200901_200005_bsize50_30eph/chkpt/trainer0/sgd.shared.training.epoch.30.step.13500/model0/'
# data_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072615_bsize256_8gpurun/chkpt/trainer0/sgd.shared.training.epoch.20.step.17580/model0/'
fldr,epoch,step='20200905_080143_bsize25_test',34,30600
data_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/{0}/chkpt/trainer0/sgd.shared.training.epoch.{1}.step.{2}/model0/'.format(fldr,epoch,step)

dict_pars['data_dir']=data_dir
dict_pars['code_dir']=code_dirs[str(img_size)]


print(dict_pars)


{'job_name': 'gen_img', 'time': '00:15:00', 'procs': 8, 'seed': 282, 'batchsize': 200, 'data_dir': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20200905_080143_bsize25_test/chkpt/trainer0/sgd.shared.training.epoch.34.step.30600/model0/', 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/512_sqr/'}


In [42]:
assert os.path.isdir(dict_pars['data_dir'])

In [43]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=8
#SBATCH --gpus-per-task=1
#SBATCH --time={time}
#SBATCH --job-name={job_name}

### Initial setup
module purge
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python3/3.7-anaconda-2019.10                                                                                                       

module load esslurm
#module load python3/3.7-anaconda-2019.10
##module use /global/cfs/cdirs/m3363/lbann/tom_lbann_install/etc/modulefiles
##module load gcc/8.2.0
##module load lbann

###unset MKL_THREADING_LAYER
export MKL_THREADING_LAYER=GNU

#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git
#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
#$SPACK_ROOT/share/spack/setup-env.sh

### Brian's installation
export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git; . $SPACK_ROOT/share/spack/setup-env.sh
spack env activate -p lbann-dev-skylake_avx512

module use /global/cfs/cdirs/m3363/lbann/lbann.git/build/gnu.Release.cgpu.nersc.gov/install/etc/modulefiles/                                                                                                       
module load lbann

export MV2_ENABLE_AFFINITY=0
export MV2_USE_CUDA=1
export IBV_FORK_SAFE=1

### Run the main code
code_dir={code_dir}
data_dir={data_dir}                                                 

python $code_dir/test_exagan.py --seed {seed} --nodes 1 --procs {procs} --epochs 1 -dr $data_dir -b {batchsize}
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [44]:
# bash_strg

In [45]:
### Write bash file
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'
fname='batch_gen_images.sh'
filename=staging_loc+fname
print(filename)
with open (filename,'w') as f:
    f.write(bash_strg)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/batch_gen_images.sh


In [46]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=8
#SBATCH --gpus-per-task=1
#SBATCH --time=00:15:00
#SBATCH --job-name=gen_img

### Initial setup
module purge
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python3/3.7-anaconda-2019.10                                                                                                       

module load esslurm
#module load python3/3.7-anaconda-2019.10
##module use /global/cfs/cdirs/m3363/lbann/tom_lbann_install/etc/modulefiles
##module load gcc/8.2.0
##module load lbann

###unset MKL_THREADING_LAYER
export MKL_THREADING_LAYER=GNU

#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git
#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
#$SPACK_ROOT/share/spack/setup-env.sh

### Brian's installation
export SPACK_ROOT=/global/cfs/cdirs/m3363/lba

In [47]:
### Move to staging locations in project space:
os.chdir(staging_loc)

### Submit jobs to cori queue

In [48]:
%%bash -s "$filename" ## Use python variable in bash
module load esslurm
sbatch $1

Submitted batch job 1010789
